In [6]:
import logging, os, sys, json
import pandas as pd

from src.utils.utils import (
    DATA_MONTH_DAY,
    SUBSCRIBER_DATA_PATH_JSON,
    SUBSCRIBER_FOLDER,
    curr_time_micro,
    sub_logger,
    lat_long_filler
)

logging.basicConfig(
        format="",
        filename=f"logs/notebook-{DATA_MONTH_DAY}.log",
        encoding="utf-8",
        filemode="a",
        level=logging.INFO,
)

In [7]:
import src.subpipe.store as store
curr_file = open(os.path.join(SUBSCRIBER_FOLDER, "04-12.json"))
json_from_file = json.load(curr_file)
db_worker = store.DataToSQLDB(json_from_file)

In [8]:
preped_df = db_worker.prepare_df(pd.DataFrame.from_dict(db_worker.json_data))

[05-09-2024-11:30:59.019] LONGITUDE GOOD! Longitude sits within -122 and -124! Min and max vals are: -123.115868, -122.372643.
[05-09-2024-11:30:59.020] LATITUDE ASSERT! Latitude sits within 45 and 46! Min and max vals are: 45.318728, 45.639137.
[05-09-2024-11:30:59.021] HDOP ASSERT GOOD! All HDOP values 4 upto (not including) 23.1 are NaN on lat and long.
[05-09-2024-11:30:59.021] MIN GPS satellites GOOD! Minimum number of satellites was: 0.0!
[05-09-2024-11:30:59.022] MAX GPS satellites GOOD! Minimum number of satellites was: 12.0!
[05-09-2024-11:30:59.022] ZERO SATELLITES ASSERT GOOD! It seems that SOME 0 GPS satellite vehicles HAVE a lat and long.
[05-09-2024-11:30:59.037] TWELVE SATELLITES ASSERT GOOD! It seems that ALL 12 GPS satellite vehicles HAVE a lat and long.
[05-09-2024-11:30:59.041] METERS RECORD ASSERT GOOD! It seems that ALL records HAVE an a meters metric.
[05-09-2024-11:30:59.041] ACTIVITY RECORD ASSERT GOOD! It seems that ALL records HAVE an event activity time.

[05

In [11]:
preped_df

,EVENT_NO_TRIP,EVENT_NO_STOP,VEHICLE_ID,METERS,SPEED,TIMESTAMP,GPS_LONGITUDE,GPS_LATITUDE
0,219000623,219000624,2905,37,2.00,2022-12-13 06:18:00,-122.843617,45.504060
1,219000623,219000624,2905,47,2.00,2022-12-13 06:18:05,-122.843743,45.504095
2,219000623,219000624,2905,54,1.40,2022-12-13 06:18:10,-122.843828,45.504118
3,219000623,219000626,2905,181,2.54,2022-12-13 06:19:00,-122.843408,45.505170
4,219000623,219000626,2905,196,3.00,2022-12-13 06:19:05,-122.843285,45.505283
...,...,...,...,...,...,...,...,...
295213,218570609,218570626,4305,190705,6.20,2022-12-13 18:15:03,-122.844028,45.504288
295214,218570609,218570626,4305,190736,6.20,2022-12-13 18:15:08,-122.844162,45.504048
295215,218570609,218570626,4305,190761,5.00,2022-12-13 18:15:13,-122.844325,45.503868
295216,218570609,218570626,4305,190784,4.60,2022-12-13 18:15:18,-122.844518,45.503735


In [18]:
df = preped_df
renamer = {"TIMESTAMP":"tstamp", "GPS_LATITUDE":"latitude", "GPS_LONGITUDE":"longitude", "SPEED":"speed", "EVENT_NO_TRIP":"trip_id"}
bc_table = df[
    ["TIMESTAMP", "GPS_LATITUDE", "GPS_LONGITUDE", "SPEED", "EVENT_NO_TRIP"]
]
bc_table.rename(columns=renamer)

SyntaxError: ':' expected after dictionary key (2631418369.py, line 2)

In [17]:
bc_table

,TIMESTAMP,GPS_LATITUDE,GPS_LONGITUDE,SPEED,EVENT_NO_TRIP
0,2022-12-13 06:18:00,45.504060,-122.843617,2.00,219000623
1,2022-12-13 06:18:05,45.504095,-122.843743,2.00,219000623
2,2022-12-13 06:18:10,45.504118,-122.843828,1.40,219000623
3,2022-12-13 06:19:00,45.505170,-122.843408,2.54,219000623
4,2022-12-13 06:19:05,45.505283,-122.843285,3.00,219000623
...,...,...,...,...,...
295213,2022-12-13 18:15:03,45.504288,-122.844028,6.20,218570609
295214,2022-12-13 18:15:08,45.504048,-122.844162,6.20,218570609
295215,2022-12-13 18:15:13,45.503868,-122.844325,5.00,218570609
295216,2022-12-13 18:15:18,45.503735,-122.844518,4.60,218570609
